In [1]:
import os
import re
import json
from pathlib import Path

import requests
import spotipy
import pandas as pd
from tqdm import tqdm
from requests.exceptions import ReadTimeout
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from dotenv import load_dotenv
from more_itertools import chunked

In [2]:
load_dotenv()

True

# Data Acquistion

In [3]:
RAW_DATA_PATH = Path('data/raw')
INTERIM_DATA_PATH = Path('data/interim')
EXTERNAL_DATA_PATH = Path('data/external')

RAW_DATA_PATH_BILLBOARD = RAW_DATA_PATH / 'billboard'
RAW_DATA_PATH_SPOTIFY = RAW_DATA_PATH / 'spotify'
INTERIM_DATA_BILLBOARD = INTERIM_DATA_PATH / 'billboard'
EXTERNAL_DATA_PATH_SPOTIFY = EXTERNAL_DATA_PATH / 'spotify'

RAW_DATA_PATH_BILLBOARD.mkdir(exist_ok=True, parents=True)
INTERIM_DATA_BILLBOARD.mkdir(exist_ok=True, parents=True)
RAW_DATA_PATH_SPOTIFY.mkdir(exist_ok=True, parents=True)
EXTERNAL_DATA_PATH_SPOTIFY.mkdir(exist_ok=True, parents=True)

In [14]:
BILLBOARD_TABLE_FILEPATH = INTERIM_DATA_BILLBOARD / "billboard_data.csv"
SONG_IDS_FILEPATH = EXTERNAL_DATA_PATH_SPOTIFY / "song_ids.csv"
SONG_AUDIO_FEATURES_FILEPATH = EXTERNAL_DATA_PATH_SPOTIFY / "song_audiofeatures.csv"

## Billboard Data

### Download

In [12]:
BILLBOARD_BASE_DOWNLOAD_URL = "https://raw.githubusercontent.com/kevinschaich/billboard/master/data/years/{}.json"
BILLBOARD_DOWNLOAD_YEARS = list(range(1950, 2015 + 1))

In [5]:
for year in tqdm(BILLBOARD_DOWNLOAD_YEARS):
    r = requests.get(BILLBOARD_BASE_DOWNLOAD_URL.format(year))
    if r.status_code != 200: print(f"Error: {year}")
    with open(RAW_DATA_PATH_BILLBOARD/"{}.json".format(year), 'wb') as f:
        f.write(r.content)

100%|██████████| 66/66 [00:44<00:00,  1.49it/s]


### Tabulating Data

In [6]:
def flatten_dict(dd, separator ='_', prefix =''): 
    return { prefix + separator + k if prefix else k : v 
             for kk, vv in dd.items() 
             for k, v in flatten_dict(vv, separator, kk).items() 
             } if isinstance(dd, dict) else { prefix : dd } 

In [7]:
years_dfs = []
for year in tqdm(BILLBOARD_DOWNLOAD_YEARS):
    with open(RAW_DATA_PATH_BILLBOARD/"{}.json".format(year), 'r') as f:
        flatten_data = [flatten_dict(x) for x in json.load(f)]
        years_dfs.append(pd.DataFrame(flatten_data))
                         
billboard_table_df = pd.concat(years_dfs, keys=BILLBOARD_DOWNLOAD_YEARS)
billboard_table_df = billboard_table_df.reset_index(level=0).rename(columns={'level_0': 'year'})
billboard_table_df = billboard_table_df.reset_index(drop=True)

print(len(billboard_table_df))
                         
billboard_table_df.to_csv(BILLBOARD_TABLE_FILEPATH) 

100%|██████████| 66/66 [00:00<00:00, 219.18it/s]


4028


## Spotify Data

In [13]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=15)

SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

In [ ]:
# TODO: Passar isso para um script
def spotify_api_get_song_id(track=None, artist=None, override_query=None):
    query = f'artist:{artist} track:{track}' if override_query is None else override_query
    
    try:
        track_id = sp.search(q=query, type='track', limit=1)
        song_info = track_id['tracks']['items'][0]
    except (KeyError, ReadTimeout, IndexError):
        song_info = {}
        
    api_song_id = song_info.get('id')
    api_song_name = song_info.get('name')
    api_song_artists = song_info.get('artists')
    
    if api_song_artists is not None: 
        api_song_artists = [artist.get('name') for artist in api_song_artists]

    return api_song_id, api_song_name, api_song_artists

def spotify_api_by_requests(query):
    
    payload = {'q': query, 'type': 'track', 'limit': 1}
    headers = {"Authorization": f"Bearer {os.getenv('SPOTIFY_OAUTH_TOKEN')}"}
    r = requests.get('https://api.spotify.com/v1/search', headers=headers, params=payload)
    try:
        song_info = r.json()['tracks']['items'][0]
    except (KeyError, IndexError):
        song_info = {}
        
    api_song_id = song_info.get('id')
    api_song_name = song_info.get('name')
    api_song_artists = song_info.get('artists')
    
    if api_song_artists is not None: 
        api_song_artists = [artist.get('name') for artist in api_song_artists]

    return api_song_id, api_song_name, api_song_artists

### Get song names and artists from billboard data

In [ ]:
songs_artists = []

for year in tqdm(BILLBOARD_DOWNLOAD_YEARS):
    with open(RAW_DATA_PATH_BILLBOARD/"{}.json".format(year), 'r') as f:
        data = json.load(f)
        year_songs_artists = [(d['title'], d['artist']) for d in data]
        songs_artists.extend(year_songs_artists)
        
len(songs_artists)

### Get song IDs from Spotify 

In [ ]:
billboard_table_df = pd.read_csv(BILLBOARD_TABLE_FILEPATH, index_col=0)
song_artists = [(value['title'], value['artist']) for index, value in billboard_table_df[['title', 'artist']].iterrows()]

spotify_ids_dict = {'name': [], 'artist': [], 'api_id': [], 'api_name': [], 'api_artists': []}
for song_name, song_artist in tqdm(songs_artists):
    
    # Base Search Songs
    base_api_music = spotify_api_get_song_id(song_name, song_artist)
    
    # Searching Without Apostrophes
    if any(x is None for x in base_api_music): 
        song_name_cleaned = song_name.replace("'", '')
        song_artist_cleaned = song_artist.replace("'", '')
        base_api_music = spotify_api_get_song_id(song_name_cleaned, song_artist_cleaned)
    
    # Searching Without Symbols
    if any(x is None for x in base_api_music):
        song_name_cleaned = re.sub(r'[-\.\+,\(\)]', "", song_name_cleaned)
        song_artist_cleaned = re.sub(r'[-\.\+,\(\)]', "", song_artist_cleaned)
        base_api_music = spotify_api_get_song_id(song_name_cleaned, song_artist_cleaned)
        
    # Wide search
    if any(x is None for x in base_api_music):
        base_api_music = spotify_api_get_song_id(override_query=" ".join([song_name_cleaned, song_artist_cleaned]))
        
    # Adding data to dataframe
    api_song_id = base_api_music[0]
    api_song_name = base_api_music[1]
    api_song_artists = ",".join(base_api_music[2]) if base_api_music[2] is not None else None

    spotify_ids_dict["name"].append(song_name)
    spotify_ids_dict["artist"].append(song_artist)
    spotify_ids_dict["api_id"].append(api_song_id)
    spotify_ids_dict["api_name"].append(api_song_name)
    spotify_ids_dict["api_artists"].append(api_song_artists)
    
spotify_ids_df = pd.DataFrame(spotify_ids_dict)
spotify_ids_df = spotify_ids_df.drop_duplicates()
spotify_ids_df.to_csv(SONG_IDS_FILEPATH)

#### Missing Values Treatment

In [ ]:
spotify_ids_df = pd.read_csv(SONG_IDS_FILEPATH, index_col=0)
missing_ids = spotify_ids_df[spotify_ids_df.api_id.isnull()]

print(len(missing_ids))
missing_ids

By looking directly on SPotify API Search API https://developer.spotify.com/console/get-search-item/ I found that some values are present but somewow the library didnt' work. So I did a simple function to use the API directly

In [ ]:
for song in tqdm(list(missing_ids.itertuples())):
    
    base_api_music = spotify_api_by_requests(" ".join([song.name, song.artist]))
    
    api_song_id = base_api_music[0]
    api_song_name = base_api_music[1]
    api_song_artists = ",".join(base_api_music[2]) if base_api_music[2] is not None else None
    
    spotify_ids_df.at[song.Index, 'api_id'] = api_song_id
    spotify_ids_df.at[song.Index, 'api_name'] = api_song_name
    spotify_ids_df.at[song.Index, 'api_artists'] = api_song_artists

Finally let's check the last missing values

In [ ]:
missing_ids = spotify_ids_df[spotify_ids_df.api_id.isnull()]

print(len(missing_ids))
missing_ids

I didn't found the correct values on spotify API, so they are really missing. Let's save the dataframe

In [ ]:
print(len(spotify_ids_df))
spotify_ids_df.to_csv(SONG_IDS_FILEPATH)

### Get Audio Features from IDs
https://developer.spotify.com/documentation/web-api/reference/#object-audiofeaturesobject

In [ ]:
spotify_ids_df = pd.read_csv(SONG_IDS_FILEPATH, index_col=0)

audio_features_list = []
search_ids = spotify_ids_df.api_id.dropna()
for songs in tqdm(list(chunked(search_ids, 80))):
    audio_features = sp.audio_features(songs)
    audio_features_list.extend(audio_features)
    
audio_features_df = pd.DataFrame([feat for feat in audio_features_list if feat != None])
audio_features_df.to_csv(SONG_AUDIO_FEATURES_FILEPATH)

In [ ]:
# Checking if there are missing values
print(len(audio_features_df))

if set(search_ids) != set(audio_features_df.id.unique()):
    print(set(search_ids) - set(audio_features_df.id.unique()))

## Merge and Exporting Data

In [15]:
FINAL_DATA_PATH = INTERIM_DATA_PATH / 'spotify_billboard_data.csv'

spotify_ids_df = pd.read_csv(SONG_IDS_FILEPATH, index_col=0)
audio_features_df = pd.read_csv(SONG_AUDIO_FEATURES_FILEPATH, index_col=0)
billboard_table_df = pd.read_csv(BILLBOARD_TABLE_FILEPATH, index_col=0)

### Merging Spotify Data

In [16]:
spotify_ids_df = spotify_ids_df.set_index("api_id")
audio_features_df = audio_features_df.set_index("id")

spotify_df = spotify_ids_df.join(audio_features_df, how='outer')
print(len(spotify_df))

3917


### Merging Spotify and Billboard Data

In [17]:
final_data_df = pd.merge(
    left=spotify_df.reset_index().dropna().drop_duplicates(), 
    right=billboard_table_df, 
    how='right', 
    left_on=['name','artist'], 
    right_on=['title','artist'],
    suffixes=('_left', '_right')
)

len(final_data_df)

4028

### Exporting Data

In [18]:
final_data_df.to_csv(FINAL_DATA_PATH)